In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM

Cloning into 'LightGBM'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 20802 (delta 37), reused 48 (delta 27), pack-reused 20724
Receiving objects: 100% (20802/20802), 16.46 MiB | 7.94 MiB/s, done.
Resolving deltas: 100% (15163/15163), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728),

In [ ]:
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import time
from lightgbm import LGBMClassifier
import lightgbm as lgbm
from sklearn.model_selection import train_test_split

In [ ]:
class Timer:    
    def __enter__(self):
        self.tick = time.time()
        return self

    def __exit__(self, *args, **kwargs):
        self.tock = time.time()
        self.elapsed = self.tock - self.tick

In [ ]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 5                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_use)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [ ]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

In [ ]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset = dataset[what_to_left]
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)

  dataset2 = dataset.drop('Time', axis = 1)
  return dataset2

In [ ]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [ ]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [ ]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def scale_to_supervised(train, n_in, dropnan=True):

    # input sequence (t-n, ... t-1)
    if dropnan:
      df = pd.DataFrame(train.iloc[:,:-1])
      df2 = pd.DataFrame(train.iloc[:,-1], index = train.index)
      cols, names = list(), list()
      future = [0]
      n_vars = 1 if type(df) is list else df.shape[1]
      n_vars2 = 1 if type(df2) is list else df2.shape[1]
      for i in range(n_in, 0, -1):
          cols.append(df.shift(i))
          names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
      # forecast sequence (t, t+1, ... t+n)
      for i in future:
          cols.append(df2.shift(-i))
          if i == 0:
              names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
          else:
              names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
      # put it all together
      agg = concat(cols, axis=1)
      agg.columns = names
      # drop rows with NaN values
      agg.dropna(inplace=True)
    else:
      df = pd.DataFrame(train)
      cols, names = list(), list()
      n_vars = 1 if type(df) is list else df.shape[1]
      for i in range(n_in, 0, -1):
          cols.append(df.shift(i))
          names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
      # forecast sequence (t, t+1, ... t+n)
      # put it all together
      agg = concat(cols, axis=1)
      agg.columns = names
      # drop rows with NaN values
      agg.dropna(inplace=True)
      #agg.fillna(0,inplace=True)
    return agg

In [ ]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)

In [ ]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [ ]:
dataset2['Goal'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'sin_time', 'cos_time','Goal']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

In [ ]:
trainset = scale_to_supervised(dataset4, n_days)
trainset

In [ ]:
set(trainset['TARGET1(t)'].values)

In [ ]:
trainset2 = trainset.iloc[:,:-1]
goals = trainset.iloc[:,-1]

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(trainset2,  goals, random_state=42, test_size=0.33)

In [ ]:
model = LGBMClassifier(learning_rate=0.03, num_leaves=64, max_depth=10, device = 'gpu', objective='multiclass', n_estimators=10000)

In [ ]:
tick = time.time()
model.fit(X_train, Y_train, eval_set=[(X_valid, Y_valid)], early_stopping_rounds=200, verbose=10)
tock = time.time()
elapsed = self.tock - self.tick


print('time elapsed(seconds) for Fitting: {}'.format(elapsed))

In [ ]:
from sklearn.externals import joblib

joblib.dump(model, 'LGBM_CLF.pkl')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['LGBM_CLF.pkl']

In [ ]:
X_train

,var1(t-240),var2(t-240),var3(t-240),var4(t-240),var5(t-240),var6(t-240),var7(t-240),var8(t-240),var1(t-239),var2(t-239),var3(t-239),var4(t-239),var5(t-239),var6(t-239),var7(t-239),var8(t-239),var1(t-238),var2(t-238),var3(t-238),var4(t-238),var5(t-238),var6(t-238),var7(t-238),var8(t-238),var1(t-237),var2(t-237),var3(t-237),var4(t-237),var5(t-237),var6(t-237),var7(t-237),var8(t-237),var1(t-236),var2(t-236),var3(t-236),var4(t-236),var5(t-236),var6(t-236),var7(t-236),var8(t-236),...,var1(t-5),var2(t-5),var3(t-5),var4(t-5),var5(t-5),var6(t-5),var7(t-5),var8(t-5),var1(t-4),var2(t-4),var3(t-4),var4(t-4),var5(t-4),var6(t-4),var7(t-4),var8(t-4),var1(t-3),var2(t-3),var3(t-3),var4(t-3),var5(t-3),var6(t-3),var7(t-3),var8(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var7(t-2),var8(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1)
29025,60.0,0.0,2.8,40.18,20.0,-0.923880,-3.826834e-01,14.0,44.0,0.0,2.7,48.59,19.0,-0.965926,-0.258819,14.0,28.0,0.0,2.4,51.79,18.0,-0.991445,-0.130526,14.0,13.0,0.0,2.1,60.95,17.0,-1.000000,-1.836970e-16,14.0,2.0,0.0,1.9,65.02,16.0,-0.991445,0.130526,14.0,...,149.0,7.0,2.0,25.54,25.0,-0.500000,-0.866025,15.0,354.0,169.0,1.9,27.12,24.0,-0.608761,-0.793353,15.0,312.0,199.0,1.8,27.62,24.0,-0.707107,-0.707107,15.0,11.0,0.0,1.6,29.35,23.0,-0.793353,-0.608761,15.0,198.0,342.0,1.4,32.28,23.0,-0.866025,-5.000000e-01,15.0
36078,51.0,919.0,0.8,62.73,4.0,-0.707107,-7.071068e-01,0.0,44.0,855.0,0.6,67.39,2.0,-0.793353,-0.608761,0.0,36.0,751.0,0.5,57.40,1.0,-0.866025,-0.500000,0.0,23.0,570.0,0.4,61.75,0.0,-0.923880,-3.826834e-01,0.0,0.0,0.0,0.4,58.75,-1.0,-0.965926,-0.258819,0.0,...,55.0,1024.0,1.9,33.87,2.0,-0.130526,-0.991445,1.0,57.0,1011.0,1.6,34.64,2.0,-0.258819,-0.965926,1.0,55.0,997.0,1.3,34.64,2.0,-0.382683,-0.923880,1.0,157.0,479.0,1.0,35.87,2.0,-0.500000,-0.866025,1.0,114.0,571.0,0.8,35.87,1.0,-0.608761,-7.933533e-01,1.0
29749,0.0,0.0,0.2,34.24,19.0,-0.991445,1.305262e-01,15.0,0.0,0.0,0.2,38.23,18.0,-0.965926,0.258819,15.0,0.0,0.0,0.2,40.75,17.0,-0.923880,0.382683,15.0,0.0,0.0,0.2,45.04,17.0,-0.866025,5.000000e-01,15.0,0.0,0.0,0.3,48.04,16.0,-0.793353,0.608761,15.0,...,158.0,365.0,4.8,35.41,20.0,-0.866025,-0.500000,16.0,128.0,306.0,4.6,37.73,19.0,-0.923880,-0.382683,16.0,89.0,239.0,4.4,43.61,18.0,-0.965926,-0.258819,16.0,50.0,146.0,4.0,49.60,16.0,-0.991445,-0.130526,16.0,12.0,0.0,3.6,57.33,15.0,-1.000000,-1.836970e-16,16.0
23217,71.0,817.0,4.2,36.25,13.0,-0.923880,-3.826834e-01,7.0,64.0,740.0,3.8,40.28,13.0,-0.965926,-0.258819,7.0,53.0,635.0,2.6,46.06,11.0,-0.991445,-0.130526,7.0,38.0,479.0,1.5,58.36,9.0,-1.000000,-1.836970e-16,7.0,26.0,0.0,1.2,66.94,8.0,-0.991445,0.130526,7.0,...,393.0,72.0,3.7,77.69,3.0,-0.500000,-0.866025,7.0,273.0,21.0,3.5,77.68,2.0,-0.608761,-0.793353,7.0,268.0,28.0,3.3,76.36,2.0,-0.707107,-0.707107,7.0,236.0,27.0,3.2,82.05,2.0,-0.793353,-0.608761,7.0,229.0,60.0,3.1,80.83,2.0,-0.866025,-5.000000e-01,7.0
41652,44.0,580.0,2.8,31.47,20.0,-1.000000,-1.836970e-16,8.0,29.0,385.0,2.7,35.72,18.0,-0.991445,0.130526,8.0,0.0,0.0,2.7,38.82,16.0,-0.965926,0.258819,8.0,0.0,0.0,3.0,41.44,15.0,-0.923880,3.826834e-01,8.0,0.0,0.0,3.3,39.83,15.0,-0.866025,0.500000,8.0,...,120.0,2.0,3.3,91.59,3.0,-0.793353,-0.608761,8.0,153.0,6.0,3.6,89.69,3.0,-0.866025,-0.500000,8.0,93.0,0.0,3.7,96.43,2.0,-0.923880,-0.382683,8.0,157.0,288.0,3.9,93.92,2.0,-0.965926,-0.258819,8.0,139.0,175.0,3.8,93.95,1.0,-0.991445,-1.305262e-01,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11524,0.0,0.0,1.8,49.47,12.0,0.500000,8.660254e-01,15.0,0.0,0.0,1.7,49.46,12.0,0.608761,0.793353,15.0,0.0,0.0,1.7,47.39,12.0,0.707107,0.707107,15.0,0.0,0.0,1.6,47.39,11.0,0.793353,6.087614e-01,15.0,0.0,0.0,1.6,47.81,11.0,0.8660

In [ ]:
df_test = []

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)



    df_test.append(testset3)

X_test = pd.concat(df_test)
X_test = X_test.iloc[:, :n_obs]
X_test.shape

In [ ]:
file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
temp = pd.read_csv(file_path)
temp

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,2.8,70.34,-6.1,0.0
1,0,0,30,0,0,2.9,71.97,-6.4,0.0
2,0,1,0,0,0,2.9,69.77,-6.7,0.0
3,0,1,30,0,0,2.9,71.95,-7.1,0.0
4,0,2,0,0,0,2.9,69.48,-7.4,0.0
...,...,...,...,...,...,...,...,...,...
331,6,21,30,0,0,1.7,43.65,-5.2,0.0
332,6,22,0,0,0,1.5,40.62,-4.9,0.0
333,6,22,30,0,0,1.5,40.62,-4.9,0.0
334,6,23,0,0,0,1.5,39.21,-4.9,0.0


In [ ]:
testset2 = drop_clms(temp)
testset2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Time,WS,DHI,DNI,RH,T,TARGET,Date
0,0.0,2.8,0,0,70.34,-6.1,0.0,0
1,0.5,2.9,0,0,71.97,-6.4,0.0,0
2,1.0,2.9,0,0,69.77,-6.7,0.0,0
3,1.5,2.9,0,0,71.95,-7.1,0.0,0
4,2.0,2.9,0,0,69.48,-7.4,0.0,0
...,...,...,...,...,...,...,...,...
331,21.5,1.7,0,0,43.65,-5.2,0.0,6
332,22.0,1.5,0,0,40.62,-4.9,0.0,6
333,22.5,1.5,0,0,40.62,-4.9,0.0,6
334,23.0,1.5,0,0,39.21,-4.9,0.0,6


In [ ]:
testset3 = cos_time(testset2)
testset3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,WS,DHI,DNI,RH,T,TARGET,Date,sin_time,cos_time
0,2.8,0,0,70.34,-6.1,0.0,0,0.000000,1.000000
1,2.9,0,0,71.97,-6.4,0.0,0,0.130526,0.991445
2,2.9,0,0,69.77,-6.7,0.0,0,0.258819,0.965926
3,2.9,0,0,71.95,-7.1,0.0,0,0.382683,0.923880
4,2.9,0,0,69.48,-7.4,0.0,0,0.500000,0.866025
...,...,...,...,...,...,...,...,...,...
331,1.7,0,0,43.65,-5.2,0.0,6,-0.608761,0.793353
332,1.5,0,0,40.62,-4.9,0.0,6,-0.500000,0.866025
333,1.5,0,0,40.62,-4.9,0.0,6,-0.382683,0.923880
334,1.5,0,0,39.21,-4.9,0.0,6,-0.258819,0.965926


In [ ]:
testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'sin_time', 'cos_time']]
testset4

,DHI,DNI,WS,RH,T,TARGET,sin_time,cos_time
0,0,0,2.8,70.34,-6.1,0.0,0.000000,1.000000
1,0,0,2.9,71.97,-6.4,0.0,0.130526,0.991445
2,0,0,2.9,69.77,-6.7,0.0,0.258819,0.965926
3,0,0,2.9,71.95,-7.1,0.0,0.382683,0.923880
4,0,0,2.9,69.48,-7.4,0.0,0.500000,0.866025
...,...,...,...,...,...,...,...,...
331,0,0,1.7,43.65,-5.2,0.0,-0.608761,0.793353
332,0,0,1.5,40.62,-4.9,0.0,-0.500000,0.866025
333,0,0,1.5,40.62,-4.9,0.0,-0.382683,0.923880
334,0,0,1.5,39.21,-4.9,0.0,-0.258819,0.965926


In [ ]:
testset5 = scale_to_supervised(testset4, n_days, dropnan=False)
testset5

,var1(t-240),var2(t-240),var3(t-240),var4(t-240),var5(t-240),var6(t-240),var7(t-240),var8(t-240),var1(t-239),var2(t-239),var3(t-239),var4(t-239),var5(t-239),var6(t-239),var7(t-239),var8(t-239),var1(t-238),var2(t-238),var3(t-238),var4(t-238),var5(t-238),var6(t-238),var7(t-238),var8(t-238),var1(t-237),var2(t-237),var3(t-237),var4(t-237),var5(t-237),var6(t-237),var7(t-237),var8(t-237),var1(t-236),var2(t-236),var3(t-236),var4(t-236),var5(t-236),var6(t-236),var7(t-236),var8(t-236),...,var1(t-5),var2(t-5),var3(t-5),var4(t-5),var5(t-5),var6(t-5),var7(t-5),var8(t-5),var1(t-4),var2(t-4),var3(t-4),var4(t-4),var5(t-4),var6(t-4),var7(t-4),var8(t-4),var1(t-3),var2(t-3),var3(t-3),var4(t-3),var5(t-3),var6(t-3),var7(t-3),var8(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var7(t-2),var8(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1)
240,0.0,0.0,2.8,70.34,-6.1,0.0,0.000000,1.000000,0.0,0.0,2.9,71.97,-6.4,0.0,0.130526,0.991445,0.0,0.0,2.9,69.77,-6.7,0.0,0.258819,0.965926,0.0,0.0,2.9,71.95,-7.1,0.0,0.382683,0.923880,0.0,0.0,2.9,69.48,-7.4,0.0,0.500000,0.866025,...,0.0,0.0,0.8,68.74,-8.1,0.0,-0.608761,0.793353,0.0,0.0,0.7,68.71,-8.5,0.0,-0.500000,0.866025,0.0,0.0,0.6,69.79,-8.7,0.0,-0.382683,0.923880,0.0,0.0,0.5,67.91,-9.0,0.0,-0.258819,0.965926,0.0,0.0,0.4,68.98,-9.2,0.0,-0.130526,0.991445
241,0.0,0.0,2.9,71.97,-6.4,0.0,0.130526,0.991445,0.0,0.0,2.9,69.77,-6.7,0.0,0.258819,0.965926,0.0,0.0,2.9,71.95,-7.1,0.0,0.382683,0.923880,0.0,0.0,2.9,69.48,-7.4,0.0,0.500000,0.866025,0.0,0.0,2.9,72.13,-7.9,0.0,0.608761,0.793353,...,0.0,0.0,0.7,68.71,-8.5,0.0,-0.500000,0.866025,0.0,0.0,0.6,69.79,-8.7,0.0,-0.382683,0.923880,0.0,0.0,0.5,67.91,-9.0,0.0,-0.258819,0.965926,0.0,0.0,0.4,68.98,-9.2,0.0,-0.130526,0.991445,0.0,0.0,1.8,57.06,-15.6,0.0,0.000000,1.000000
242,0.0,0.0,2.9,69.77,-6.7,0.0,0.258819,0.965926,0.0,0.0,2.9,71.95,-7.1,0.0,0.382683,0.923880,0.0,0.0,2.9,69.48,-7.4,0.0,0.500000,0.866025,0.0,0.0,2.9,72.13,-7.9,0.0,0.608761,0.793353,0.0,0.0,2.9,69.76,-8.3,0.0,0.707107,0.707107,...,0.0,0.0,0.6,69.79,-8.7,0.0,-0.382683,0.923880,0.0,0.0,0.5,67.91,-9.0,0.0,-0.258819,0.965926,0.0,0.0,0.4,68.98,-9.2,0.0,-0.130526,0.991445,0.0,0.0,1.8,57.06,-15.6,0.0,0.000000,1.000000,0.0,0.0,1.5,57.06,-15.6,0.0,0.130526,0.991445
243,0.0,0.0,2.9,71.95,-7.1,0.0,0.382683,0.923880,0.0,0.0,2.9,69.48,-7.4,0.0,0.500000,0.866025,0.0,0.0,2.9,72.13,-7.9,0.0,0.608761,0.793353,0.0,0.0,2.9,69.76,-8.3,0.0,0.707107,0.707107,0.0,0.0,2.9,71.97,-8.7,0.0,0.793353,0.608761,...,0.0,0.0,0.5,67.91,-9.0,0.0,-0.258819,0.965926,0.0,0.0,0.4,68.98,-9.2,0.0,-0.130526,0.991445,0.0,0.0,1.8,57.06,-15.6,0.0,0.000000,1.000000,0.0,0.0,1.5,57.06,-15.6,0.0,0.130526,0.991445,0.0,0.0,1.3,56.57,-15.5,0.0,0.258819,0.965926
244,0.0,0.0,2.9,69.48,-7.4,0.0,0.500000,0.866025,0.0,0.0,2.9,72.13,-7.9,0.0,0.608761,0.793353,0.0,0.0,2.9,69.76,-8.3,0.0,0.707107,0.707107,0.0,0.0,2.9,71.97,-8.7,0.0,0.793353,0.608761,0.0,0.0,2.9,68.56,-9.1,0.0,0.866025,0.500000,...,0.0,0.0,0.4,68.98,-9.2,0.0,-0.130526,0.991445,0.0,0.0,1.8,57.06,-15.6,0.0,0.000000,1.000000,0.0,0.0,1.5,57.06,-15.6,0.0,0.130526,0.991445,0.0,0.0,1.3,56.57,-15.5,0.0,0.258819,0.965926,0.0,0.0,1.0,56.11,-15.4,0.0,0.382683,0.923880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,0.0,0.0,1.3,60.39,-9.8,0.0,-0.608761,0.793353,0.0,0.0,1.2,61.04,-9.9,0.0,-0.500000,0.866025,0.0,0.0,1.1,62.02,-10.1,0.0,-0.382683,0.923880,0.0,0.0,0.9,62.31,-10.2,0.0,-0.258819,0.965926,0.0,0.0,0.8,63.81,-10.5,0.0,-0.130526,0.991445,...,0.0,0.0,1.5,41.92,-4.9,0.0,-0.965926,0.258819,0.0,0.0,1.5,42.88,-5.2,0.0,-0.923880,0.382683,0.0,0.0,1.6,45.58,-5.4,0.0,-0.866025,0.500000,0.0,0.0,1.7,45.58,-5.4,0.0,-0.793353,0.608761,0.0,0.0,1.8,44.37,-5.4,0.0,-0.707107,0.707107
3